In [1]:
import time
start = time.time()
import random
import string
import pandas as pd
import copy
#import os
#os.chdir('/home/scott/Documents/EDx/MIT/Python/Spyder_scripts/')
WORDLIST_FILENAME = "words.txt"

def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # line: string
    line = inFile.readline()
    # wordlist: list of strings
    wordlist = line.split()
    print("  ", len(wordlist), "words loaded.")
    return wordlist

def chooseWord(wordlist):
    """
    wordlist (list): list of words (strings)

    Returns a word from wordlist at random
    """
    return random.choice(wordlist)

# end of helper code
# -----------------------------------

# Load the list of words into the variable wordlist
# so that it can be accessed from anywhere in the program

def isWordGuessed(secretWord, lettersGuessed):
    '''
    secretWord: string, the word the user is guessing
    lettersGuessed: list, what letters have been guessed so far
    returns: boolean, True if all the letters of secretWord are in lettersGuessed;
      False otherwise
    '''
    for char in secretWord:
        if char not in lettersGuessed:
            return False
    
    return True  


def getGuessedWord(secretWord, lettersGuessed):
    '''
    secretWord: string, the word the user is guessing
    lettersGuessed: list, what letters have been guessed so far
    returns: string, comprised of letters and underscores that represents
      what letters in secretWord have been guessed so far.
    '''
    # FILL IN YOUR CODE HERE...
    word = secretWord
 
    for char in word:
        if char not in lettersGuessed:
            word = word.replace(char,'_')
            
    return word
    


def getAvailableLetters(lettersGuessed):
    '''
    lettersGuessed: list, what letters have been guessed so far
    returns: string, comprised of letters that represents what letters have not
      yet been guessed.
    '''
    # FILL IN YOUR CODE HERE...
    availableLetters = 'abcdefghijklmnopqrstuvwxyz'
    for char in lettersGuessed:
            position = availableLetters.index(char)
            availableLetters = availableLetters[0:position] + availableLetters[position+1::]
    return availableLetters

wordList = loadWords()
columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13']
secrets = pd.DataFrame()
    
for i in range(1,max(len(word) for word in wordList)+1):
    dat = []
    for word in wordList:
        if len(word) == i:
            dat.append(word)
    dat = pd.Series(dat)
    secrets = pd.concat([secrets,dat],axis=1, join='outer', join_axes=None, ignore_index=False)
secrets.columns = columns

def hangman():
    '''
    secretWord: string, the secret word to guess.

    Starts up an interactive game of Hangman.

    * At the start of the game, let the user know how many 
      letters the secretWord contains.

    * Ask the user to supply one guess (i.e. letter) per round.

    * The user should receive feedback immediately after each guess 
      about whether their guess appears in the computers word.

    * After each round, you should also display to the user the 
      partially guessed word so far, as well as letters that the 
      user has not yet guessed.

    Follows the other limitations detailed in the problem write-up.
    '''
    
    n = 1
    gamesWon = 0
    while n <= 100:    
        secretWord = chooseWord(wordList)
        word = secretWord
        choices = secrets[str(len(word))]
        numberOfGuesses = 8
        print('Welcome to the game Hangman!')
        print('I am thinking of a word that is ' + str(len(word)) + ' letters long.')
        print('-------------')
        lettersGuessed = ''
        game = True
        
        while game == True:
            
                
            availableLetters = getAvailableLetters(lettersGuessed)
            print('You have ' + str(numberOfGuesses) + ' guesses left.')
            print('Available letters: ' + availableLetters)
                
            word = getGuessedWord(secretWord, lettersGuessed)
            #letterGuessed = player(secretWord, lettersGuessed)
            [letterGuessed,words] = playerAI(secretWord, lettersGuessed,word,choices)
            choices = words
                
            lettersGuessed = lettersGuessed[0::] + letterGuessed
            word = getGuessedWord(secretWord, lettersGuessed)
                    
            if letterGuessed not in secretWord:
                print("Oops! That letter is not in my word: " + word)
                print('-------------')
                numberOfGuesses -= 1
                    
            else:
                print('Good guess: ' + word)
                print('-------------')
               
            answer = isWordGuessed(secretWord,lettersGuessed)
        
            if numberOfGuesses == 0 or answer == True:
                game = False    
            
            
        if answer == True:
            print('Congratulations, you won!')
            gamesWon += 1
            n += 1
            
        else:
            print('Sorry, you ran out of guesses. The word was ' + secretWord)
            n += 1
            
    print('Player function won ' + str(gamesWon) + ' games.')
    print('Player function lost ' + str(n - gamesWon - 1) + ' games.')
    return n

# When you've completed your hangman function, uncomment these two lines
# and run this file to test! (hint: you might want to pick your own
# secretWord while you're testing)

def playerAI(secretWord, lettersGuessed,word, choices):

    availableLetters = getAvailableLetters(lettersGuessed)
    col = str(len(secretWord))
    words = secrets[col].dropna()
    words = list(words)
    options = copy.deepcopy(words)

    if len(availableLetters) < 26:
        for chance in words:
            for j in range(0,len(chance)):
                if word[j] != '_' and  word[j] != chance[j]:
                    options.remove(chance)
                    break       
            
    best = 0
    choice = ''
    for letter in availableLetters:
        
        maximum = 0
        for word in options:
            for char in word:
                if char == letter:
                    maximum += 1
        if maximum > best:
            best = maximum
            choice = letter

    return [choice,options]
    

def player(secretWord, lettersGuessed):
    
    index = random.choice(range(0,len(string.ascii_lowercase) - len(lettersGuessed)  , 1))
    availableLetters = getAvailableLetters(lettersGuessed)
    letter = availableLetters[index]
    
    return letter

n = hangman()
end = time.time()
print('PlayerAI played ' + str(n-1) + ' games in '+ str(round((end-start)/60,2)) + ' minutes')

Loading word list from file...
('  ', 55909, 'words loaded.')
Welcome to the game Hangman!
I am thinking of a word that is 5 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Oops! That letter is not in my word: _____
-------------
You have 7 guesses left.
Available letters: abcdefghijklmnopqrtuvwxyz
Oops! That letter is not in my word: _____
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Oops! That letter is not in my word: _____
-------------
You have 5 guesses left.
Available letters: bcdfghijklmnopqrtuvwxyz
Good guess: __r__
-------------
You have 5 guesses left.
Available letters: bcdfghijklmnopqtuvwxyz
Good guess: _or__
-------------
You have 5 guesses left.
Available letters: bcdfghijklmnpqtuvwxyz
Good guess: _ort_
-------------
You have 5 guesses left.
Available letters: bcdfghijklmnpquvwxyz
Good guess: fort_
-------------
You have 5 guesses left.
Available letters: bcdghijklmnpquvwxyz
Oops! Th

Good guess: ___i_i____
-------------
You have 7 guesses left.
Available letters: abcdfghjklmnopqrstuvwxyz
Oops! That letter is not in my word: ___i_i____
-------------
You have 6 guesses left.
Available letters: abcdfghjklmnopqrsuvwxyz
Good guess: ___i_in___
-------------
You have 6 guesses left.
Available letters: abcdfghjklmopqrsuvwxyz
Oops! That letter is not in my word: ___i_in___
-------------
You have 5 guesses left.
Available letters: abcdfghjklmopqruvwxyz
Good guess: ___i_in_l_
-------------
You have 5 guesses left.
Available letters: abcdfghjkmopqruvwxyz
Good guess: ___i_ingl_
-------------
You have 5 guesses left.
Available letters: abcdfhjkmopqruvwxyz
Good guess: ___i_ingly
-------------
You have 5 guesses left.
Available letters: abcdfhjkmopqruvwxz
Good guess: ___iringly
-------------
You have 5 guesses left.
Available letters: abcdfhjkmopquvwxz
Good guess: a__iringly
-------------
You have 5 guesses left.
Available letters: bcdfhjkmopquvwxz
Good guess: ad_iringly
---------

Good guess: ______its
-------------
You have 5 guesses left.
Available letters: bcdfghjklmnopquvwxyz
Oops! That letter is not in my word: ______its
-------------
You have 4 guesses left.
Available letters: bcdfghjklmnoquvwxyz
Good guess: _____nits
-------------
You have 4 guesses left.
Available letters: bcdfghjklmoquvwxyz
Good guess: ___d_nits
-------------
You have 4 guesses left.
Available letters: bcfghjklmoquvwxyz
Good guess: _h_d_nits
-------------
You have 4 guesses left.
Available letters: bcfgjklmoquvwxyz
Good guess: _hod_nits
-------------
You have 4 guesses left.
Available letters: bcfgjklmquvwxyz
Good guess: _hodunits
-------------
You have 4 guesses left.
Available letters: bcfgjklmqvwxyz
Good guess: whodunits
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 5 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Oops! That letter is not in my word: _____
-------------
You have

Oops! That letter is not in my word: ________
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: ________
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: ___i_i__
-------------
You have 6 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Good guess: ___i_in_
-------------
You have 6 guesses left.
Available letters: abcdfghjklmopqrtuvwxyz
Good guess: ___i_ing
-------------
You have 6 guesses left.
Available letters: abcdfhjklmopqrtuvwxyz
Oops! That letter is not in my word: ___i_ing
-------------
You have 5 guesses left.
Available letters: abcdfhjklmopqtuvwxyz
Oops! That letter is not in my word: ___i_ing
-------------
You have 4 guesses left.
Available letters: abcdfhjklmopquvwxyz
Good guess: __ai_ing
-------------
You have 4 guesses left.
Available letters: bcdfhjklmopquvwxyz
Good guess: _lai_ing
-------------
You have 4 guesses left.
Available letters: bcdfhjkmo

Good guess: fle_time
-------------
You have 2 guesses left.
Available letters: bdghjknpquwxyz
Good guess: flextime
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 7 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: ___e___
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: s__e___
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: s_re___
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqtuvwxyz
Good guess: stre___
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopquvwxyz
Good guess: strea__
-------------
You have 8 guesses left.
Available letters: bcdfghijklmnopquvwxyz
Good guess: streak_
-------------
You have 8 guesses left.
Available letters: bcdfghijlmnopquvwxyz
Good guess: streaky
-------------
Congratulations, you won!
Welcome to the

...

Oops! That letter is not in my word: _____
-------------
You have 7 guesses left.
Available letters: abcdefghijklmnopqrtuvwxyz
Good guess: ___e_
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: __rer
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqtuvwxyz
Oops! That letter is not in my word: __rer
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnpqtuvwxyz
Oops! That letter is not in my word: __rer
-------------
You have 5 guesses left.
Available letters: bcdfghijklmnpqtuvwxyz
Oops! That letter is not in my word: __rer
-------------
You have 4 guesses left.
Available letters: cdfghijklmnpqtuvwxyz
Good guess: _irer
-------------
You have 4 guesses left.
Available letters: cdfghjklmnpqtuvwxyz
Good guess: direr
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 6 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmn

Good guess: ________e
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: _____a__e
-------------
You have 8 guesses left.
Available letters: bcdfghijklmnopqrstuvwxyz
Good guess: _____a_le
-------------
You have 8 guesses left.
Available letters: bcdfghijkmnopqrstuvwxyz
Good guess: _____able
-------------
You have 8 guesses left.
Available letters: cdfghijkmnopqrstuvwxyz
Oops! That letter is not in my word: _____able
-------------
You have 7 guesses left.
Available letters: cdfghijkmnopqstuvwxyz
Good guess: i____able
-------------
You have 7 guesses left.
Available letters: cdfghjkmnopqstuvwxyz
Oops! That letter is not in my word: i____able
-------------
You have 6 guesses left.
Available letters: cdfghjkmopqstuvwxyz
Good guess: imm__able
-------------
You have 6 guesses left.
Available letters: cdfghjkopqstuvwxyz
Good guess: immo_able
-------------
You have 6 guesses left.
Available letters: cdfghjkpqstuvwxyz
Good guess: immovable
----------

Good guess: ________e_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: s_______e_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Oops! That letter is not in my word: s_______e_
-------------
You have 7 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Oops! That letter is not in my word: s_______e_
-------------
You have 6 guesses left.
Available letters: abcdfghjklmnopqruvwxyz
Good guess: s_r___r_er
-------------
You have 6 guesses left.
Available letters: abcdfghjklmnopquvwxyz
Good guess: s_r__ar_er
-------------
You have 6 guesses left.
Available letters: bcdfghjklmnopquvwxyz
Good guess: s_rc_ar_er
-------------
You have 6 guesses left.
Available letters: bdfghjklmnopquvwxyz
Good guess: s_rc_arger
-------------
You have 6 guesses left.
Available letters: bdfhjklmnopquvwxyz
Good guess: s_rcharger
-------------
You have 6 guesses left.
Available letters: bdfjklmnopquvwxyz
Good guess: surcharger


Oops! That letter is not in my word: _____
-------------
You have 7 guesses left.
Available letters: abcdefghijklmnopqrtuvwxyz
Good guess: _e_e_
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Oops! That letter is not in my word: _e_e_
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnopqtuvwxyz
Good guess: de_ed
-------------
You have 6 guesses left.
Available letters: abcfghijklmnopqtuvwxyz
Good guess: dewed
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 7 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: ___e_e_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: ___e_e_
-------------
You have 7 guesses left.
Available letters: abcfghijklmnopqrstuvwxyz
Good guess: _r_e_er
-------------
You have 7 guesses left.
Available letters: abcfghijklmnopqstu

Good guess: ra___ned
-------------
You have 5 guesses left.
Available letters: bcfghjklmopquvwxyz
Good guess: ra__oned
-------------
You have 5 guesses left.
Available letters: bcfghjklmpquvwxyz
Good guess: ra_boned
-------------
You have 5 guesses left.
Available letters: cfghjklmpquvwxyz
Good guess: rawboned
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 7 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: _e_____
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: _ess___
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: _essi__
-------------
You have 8 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Good guess: messi__
-------------
You have 8 guesses left.
Available letters: abcdfghjklnopqrtuvwxyz
Good guess: messia_
-------------
You have 8 guesses left.
Available letters